# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES:

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/workspace/home
['/workspace/home/event_data/2018-11-10-events.csv', '/workspace/home/event_data/2018-11-04-events.csv', '/workspace/home/event_data/2018-11-23-events.csv', '/workspace/home/event_data/2018-11-03-events.csv', '/workspace/home/event_data/2018-11-28-events.csv', '/workspace/home/event_data/2018-11-14-events.csv', '/workspace/home/event_data/2018-11-22-events.csv', '/workspace/home/event_data/2018-11-15-events.csv', '/workspace/home/event_data/2018-11-01-events.csv', '/workspace/home/event_data/2018-11-06-events.csv', '/workspace/home/event_data/2018-11-19-events.csv', '/workspace/home/event_data/2018-11-18-events.csv', '/workspace/home/event_data/2018-11-16-events.csv', '/workspace/home/event_data/2018-11-07-events.csv', '/workspace/home/event_data/2018-11-25-events.csv', '/workspace/home/event_data/2018-11-21-events.csv', '/workspace/home/event_data/2018-11-17-events.csv', '/workspace/home/event_data/2018-11-08-events.csv', '/workspace/home/event_data/2018-11-12-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))
# to check to see what the list of event data rows will look like
# display(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# checking for duplicates in the data
df = pd.read_csv('event_datafile_new.csv')
df.duplicated().any()

False

In [6]:
display(df)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Hoobastank,Cierra,F,0,Finley,241.39710,free,"Richmond, VA",132,Say The Same,96
1,Mark Knopfler,Cierra,F,1,Finley,249.31220,free,"Richmond, VA",132,Why Aye Man,96
2,Mogwai,Cierra,F,2,Finley,341.28934,free,"Richmond, VA",132,We're No Here,96
3,The Casualties,Cierra,F,3,Finley,181.49832,free,"Richmond, VA",132,Punx Unite,96
4,The Living End,Ryan,M,0,Smith,188.62975,free,"San Jose-Sunnyvale-Santa Clara, CA",433,Roll On (Album Version),26
5,Aloe Blacc,Rylan,M,0,George,244.19220,free,"Birmingham-Hoover, AL",402,I Need A Dollar,16
6,Faith No More,Rylan,M,1,George,326.50404,free,"Birmingham-Hoover, AL",402,Helpless,16
7,Chris Cornell,Aleena,F,0,Kirby,353.69751,paid,"Waterloo-Cedar Falls, IA",350,Sunshower (Great Expectations Soundtrack),44
8,Weezer,Aleena,F,1,Kirby,203.93751,paid,"Waterloo-Cedar Falls, IA",350,My Name Is Jonas,44
9,Stream of Passion feat. Ayreon,Aleena,F,2,Kirby,257.56689,paid,"Waterloo-Cedar Falls, IA",350,Valley Of The Queens,44


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating Apache Cassandra Tables in the cells below

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1} """)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Setting KEYSPACE
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Creating queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

We are basically looking for something like

`select artist, songtitle, song_length from sparkify_songs_summary where sessionid=338 and itemInSession=4`

### Justification
Here are primary key will be sessionid and clustering key will be itemInsession so we can extract unique records based on the query to pull specific sessionids and itemInSession
With the idea that the table is bigger in size than what we did in exercises and it can be bigger in production or real life scenarios, we will only create the table with the necessary columns for the data we want to show, so that the cluster is not having unnessary data to select and pull thereby saving on cluster space and retrival time.

In [10]:
## The final output should look like the below, vizulaizing it using the dataframe we created above
# print("Showing how the output should look like")
# display(df[(df['sessionId'] == 338) & (df['itemInSession'] == 4)][['artist', 'song', 'length']].style.hide(axis="index"))


# Creating the table in Apache Cassandra

create_question1 = "CREATE TABLE IF NOT EXISTS sparkify_songs_summary "
create_question1 = create_question1 + "(artist text, song_title text, song_length float, sessionId int, itemInSession int,\
                PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(create_question1)
    print("\nTable sparkify_songs_summary has been created")
except Exception as e:
    print(e)


Table sparkify_songs_summary has been created


In [11]:
# to correctly get the index for column names to reference in the csv
print(df.columns[3])

itemInSession


In [12]:
# reading from the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_songs_summary (artist, song_title, song_length, sessionId, itemInSession)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## Validating the results

query = "SELECT artist, song_title, song_length from sparkify_songs_summary WHERE sessionid=338 and itemInSession=4"
try:
    rows = session.execute(query)
    
    # Create a PrettyTable object
    table = PrettyTable()
    
    # Define the column names
    table.field_names = ["Artist", "song_title", "song_length"]
    
    # Add rows to the table
    for row in rows:
        table.add_row([row.artist, row.song_title, row.song_length])
    
    # Print the table
    print(table)
except Exception as e:
    print(e)
    


+-----------+---------------------------------+--------------------+
|   Artist  |            song_title           |    song_length     |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+



### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
We are basically looking for something like

`select artist_name, song_title, artist_firstname, artist_lastname from sparkify_artist_summary where userid=10 and sessionid=182`

### Justification
Here are primary key will be userid and clustering key will be sessionId and itemInsession so we can extract unique records based on the query and sort it by itemInsession
With the idea that the table is bigger in size than what we did in exercises and it can be bigger in production or real life scenarios, we will only create the table with the necessary columns for the data we want to show, so that the cluster is not having unnessary data to select and pull thereby saving on cluster space and retrival time.

In [14]:
# Creating the table in Apache Cassandra

create_question2 = "CREATE TABLE IF NOT EXISTS sparkify_artist_summary "
create_question2 = create_question2 + "(artist text, song_title text, artist_firstname text, artist_lastname text, \
                userid int, sessionid int, itemInSession int,\
                PRIMARY KEY (userid, sessionid, itemInSession))"
try:
    session.execute(create_question2)
    print("\nTable sparkify_artist_summary has been created")
except Exception as e:
    print(e)


Table sparkify_artist_summary has been created


In [15]:
# reading from the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_artist_summary (artist, song_title, artist_firstname, artist_lastname, \
                userid , sessionid , itemInSession)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3]) ))

In [16]:
## Validating the results
    
query = "SELECT artist, song_title, artist_firstname, artist_lastname FROM sparkify_artist_summary \
         WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query)
    
    # Create a PrettyTable object
    table = PrettyTable()
    
    # Define the column names
    table.field_names = ["Artist", "song_title", "artist_firstname", "artist_lastname"]
    
    # Add rows to the table
    for row in rows:
        table.add_row([row.artist, row.song_title, row.artist_firstname, row.artist_lastname])
    
    # Print the table
    print(table)
except Exception as e:
    print(e)


+-------------------+------------------------------------------------------+------------------+-----------------+
|       Artist      |                      song_title                      | artist_firstname | artist_lastname |
+-------------------+------------------------------------------------------+------------------+-----------------+
|  Down To The Bone |                  Keep On Keepin' On                  |      Sylvie      |       Cruz      |
|    Three Drives   |                     Greece 2000                      |      Sylvie      |       Cruz      |
| Sebastien Tellier |                      Kilometer                       |      Sylvie      |       Cruz      |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |      Sylvie      |       Cruz      |
+-------------------+------------------------------------------------------+------------------+-----------------+


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

We are basically looking for something like

`select artist_firstname, artist_lastname, song from sparkify_artist_summary where song='All Hands Against His Own'`

### Justification
Here are primary key will be song and clustering key will be userid so we can extract unique records based on the query as song is repeating in the dataset and cant be used along as primary key (shown in the below cell)
With the idea that the table is bigger in size than what we did in exercises and it can be bigger in production or real life scenarios, we will only create the table with the necessary columns for the data we want to show, so that the cluster is not having unnessary data to select and pull thereby saving on cluster space and retrival time.

In [17]:
df[df[['userId', 'song']].duplicated(keep=False)].sort_values(by='song')

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4472,The Crests,Kate,F,51,Harrell,182.88281,paid,"Lansing-East Lansing, MI",537,16 Candles,97
3343,The Crests,Kate,F,28,Harrell,182.88281,paid,"Lansing-East Lansing, MI",817,16 Candles,97
3071,Sam Cooke,Kate,F,26,Harrell,122.04363,paid,"Lansing-East Lansing, MI",797,Ain't Misbehavin,97
690,Sam Cooke,Kate,F,4,Harrell,122.04363,paid,"Lansing-East Lansing, MI",944,Ain't Misbehavin,97
4298,Sam Cooke,Kate,F,6,Harrell,122.04363,paid,"Lansing-East Lansing, MI",508,Ain't Misbehavin,97
1065,Green Day,Tegan,F,81,Levine,185.12934,paid,"Portland-South Portland, ME",548,American Idiot [feat. Green Day & The Cast Of ...,80
1020,Green Day,Tegan,F,59,Levine,185.12934,paid,"Portland-South Portland, ME",548,American Idiot [feat. Green Day & The Cast Of ...,80
5470,Dakota Oak,Tegan,F,24,Levine,45.63546,paid,"Portland-South Portland, ME",416,Baker's Blue Jay Yarn,80
966,Dakota Oak,Tegan,F,19,Levine,45.63546,paid,"Portland-South Portland, ME",548,Baker's Blue Jay Yarn,80
4177,Panic At The Disco,Kate,F,32,Harrell,259.29098,paid,"Lansing-East Lansing, MI",147,Behind The Sea [Live In Chicago],97


In [18]:
# Creating the table in Apache Cassandra

create_question3 = "CREATE TABLE IF NOT EXISTS sparkify_artist_songs "
create_question3 = create_question3 + "(artist_firstname text, artist_lastname text, song text, userid int\
                , PRIMARY KEY (song, userid))"
try:
    session.execute(create_question3)
    print("\nTable sparkify_artist_songs has been created")
except Exception as e:
    print(e)


Table sparkify_artist_songs has been created


In [19]:
# reading from the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkify_artist_songs (artist_firstname , artist_lastname , song, userid)"
        query = query + "VALUES (%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1], line[4], line[9],  int(line[10]) ))

In [20]:
## Validating the results
    
query = "SELECT artist_firstname , artist_lastname FROM sparkify_artist_songs \
         WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
    
    # Create a PrettyTable object
    table = PrettyTable()
    
    # Define the column names
    table.field_names = ["artist_firstname", "artist_lastname"]
    
    # Add rows to the table
    for row in rows:
        table.add_row([row.artist_firstname, row.artist_lastname])
    
    # Print the table
    print(table)
except Exception as e:
    print(e)


+------------------+-----------------+
| artist_firstname | artist_lastname |
+------------------+-----------------+
|    Jacqueline    |      Lynch      |
|      Tegan       |      Levine     |
|       Sara       |     Johnson     |
+------------------+-----------------+


### Drop the tables before closing out the sessions

In [21]:
## Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS sparkify_songs_summary")
    session.execute("DROP TABLE IF EXISTS sparkify_artist_summary")
    session.execute("DROP TABLE IF EXISTS sparkify_artist_songs")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()